In [ ]:
# ライブラリのimport
import sounddevice as sd
# sounddeviceはデータをnumpyで取り扱うため、import必須
import numpy as np

####
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt

In [ ]:
# デバイス一覧およびIDを取得
# 使用するデバイスを指定するためにまず使用可能なデバイス情報を取得して調べる
device_list = sd.query_devices()
print(device_list)

In [ ]:
#### 初期設定(ここで設定せず都度設定しても良い)
sd.default.device = 17 # 使用するデバイスのID
sd.default.samplerate = 48000 # # 入出力に使用されるサンプルレート
sd.default.channels = 1 # 入出力チャンネル数の数
sd.default.dtype = 'float32' # 入出力に使用されるサンプルのデータ型 defaultはfloat32

In [ ]:
# 全ての設定をリセット
sd.default.reset()

In [ ]:
#### サンプルプログラム
# 5秒間録音してプロットする apiはasio使用

# 記録する秒数
duration = 5
# サンプルレート
fs = 48000
# チャンネル数
ch = 2

# Asioデバイスの設定
asio_in = sd.AsioSettings(channel_selectors = [0,1])

# 録音開始
recdata = sd.rec(int(duration * fs),
                samplerate = fs,channels = ch, extra_settings=asio_in)

# 録音が完了するまで以後の処理を待機
sd.wait()

# グラフの描画
length = int(1000 * fs / (1000 * duration))
fig, ax = plt.subplots()
line = ax.plot(recdata)
ax.set_ylim([-1.0, 1.0])
ax.yaxis.grid(True)
fig.tight_layout()
plt.show()

In [ ]:
# 記録する秒数
duration = 5
# サンプルレート
fs = 44100
# チャンネル数
ch = 2

# 使用するデバイスのID
sd.default.device = 5
# 入出力のデータ型
sd.default.dtype = 'float32'

# Asioデバイスの設定
asio_in = sd.AsioSettings(channel_selectors = [0,1])

def callback(indata, frames, time, status):
    print(indata.shape)

with sd.InputStream(
        channels = ch,
        callback = callback
    ):
    sd.sleep(int(duration * 1000))

In [ ]:
#### サンプルプログラム
# 5秒間録音してプロットする apiはasio使用
# UMC202HDの2-inputのマイク入力波形を録音してプロットする

sd.default.device = 5
# 記録する秒数
duration = 5
# サンプルレート
fs = 44100
# チャンネル数
ch = 2
# 入出力のデータ型
sd.default.dtype = 'float32'

# Asioデバイスの設定
asio_in = sd.AsioSettings(channel_selectors = [0,1])

# 録音開始
recdata = sd.rec(int(duration * fs),
                samplerate = fs,channels = ch, extra_settings = asio_in)

# 録音が完了するまで以後の処理を待機
sd.wait()

print(recdata.shape)

# グラフ描画
fig = plt.figure()
length = int(fs * duration)
label = np.linspace(0, duration, length)
ax1 = fig.add_subplot(2, 1, 1) # input1
ax2 = fig.add_subplot(2, 1, 2) # input2
plotdata = np.zeros((length))
line1, = ax1.plot(label, recdata[:,0])
line2, = ax2.plot(label, recdata[:,1])
ax1.set_ylim([-1.0, 1.0])
ax2.set_ylim([-1.0, 1.0])
ax1.set_xlim([0, duration])
ax2.set_xlim([0, duration])
ax1.yaxis.grid(True)
ax2.yaxis.grid(True)
fig.tight_layout()
plt.show()


In [ ]:
#### サンプルプログラム
# アプリのGUIでAPIを選べるようにするために、ASIOやMMEといったAPIとInput/Outputごとにデバイス情報を保持する辞書型変数を作る

di = {}
for hostapi in sd.query_hostapis():
    hostapi_name = hostapi["name"]
    di[hostapi_name] = {"inputs": [], "outputs": []}
    for device_numbar in hostapi["devices"]:
        device = sd.query_devices(device = device_numbar)
        max_in_ch  = device["max_input_channels"]
        max_out_ch = device["max_output_channels"]

        if max_in_ch > 0:
            di[hostapi_name]["inputs"].append(device)

        if max_out_ch > 0:
            di[hostapi_name]["outputs"].append(device)
di["ASIO"]